# BBM-BBM system

In [ ]:
include("setup.jl")

# Conserving the cubic energy

In [ ]:
function relaxation_functional(γ, ηunew, ηuold, param)
    @unpack D, tmp1 = param
    ηnew = ηunew.x[1]
    unew = ηunew.x[2]
    ηold = ηuold.x[1]
    uold = ηuold.x[2]
    
    @. tmp1 = ((1-γ)*ηold + γ*ηnew)^2 + ((1-γ)*ηold + γ*ηnew + 1) * ((1-γ)*uold + γ*unew)^2
    energy = integrate(tmp1, D)
end
function relaxation_functional(ηu, param)
    @unpack D, tmp1 = param
    η = ηu.x[1]
    u = ηu.x[2]
    
    @. tmp1 = η^2 + (η + 1) * u^2
    integrate(tmp1, D)
end


function save_func_bbm_bbm_periodic(ηu, t, integrator)
    @unpack D, D2, tmp1 = integrator.p
    η = ηu.x[1]
    u = ηu.x[2]
    print(".")
    
    mass_η = integrate(η, D)
    mass_u = integrate(u, D)
    @. tmp1 = η^2 + (1 + η) * u^2
    energy = integrate(tmp1, D)
    
    mul!(tmp1, D2, u)
    @. tmp1 = η * (u - tmp1)
    quadratic = integrate(tmp1, D)
    
    x = grid(D)
    tmp1 .= ( ηsol.(t, x, x[1], -x[1]) .- η ).^2
    error_η = integrate(tmp1, D) |> sqrt
    tmp1 .= ( usol.(t, x, x[1], -x[1]) .- u ).^2
    error_u = integrate(tmp1, D) |> sqrt
    
    SVector(mass_η, mass_u, energy, quadratic, error_η, error_u)
end


function bbm_bbm_periodic!(dηu, ηu, param, t)
    @unpack D, invImD2, D_ImD2, tmp1, tmp2 = param
    η = ηu.x[1]
    u = ηu.x[2]
    dη = dηu.x[1]
    du = dηu.x[2]
    
    # energy and mass conservative semidiscretization
    @. tmp1 = -(u + η * u)
    if D_ImD2 === nothing
        ldiv!(tmp2, invImD2, tmp1)
        mul!(dη, D, tmp2)
    else
        mul!(dη, D_ImD2, tmp1)
    end

    @. tmp1 = -(η + 0.5 * u^2)
    if D_ImD2 === nothing
        ldiv!(tmp2, invImD2, tmp1)
        mul!(du, D, tmp2)
    else
        mul!(du, D_ImD2, tmp1)
    end
    
    nothing
end

function solve_ode_bbm_bbm_periodic(D, D2, tspan, alg, tol, dt, adaptive)
    invImD2 = isa(D2, AbstractMatrix) ? factorize(I - D2) : I - D2
    D_ImD2 = if isa(D2, AbstractMatrix) 
        nothing
    else
        D / (I - D2)
    end
    
    x = grid(D)
    η0 = ηsol.(tspan[1], x, x[1], -x[1])
    u0 = usol.(tspan[1], x, x[1], -x[1])
    ηu0 = ArrayPartition(η0, u0)
    tmp1 = similar(u0); tmp2 = similar(tmp1)
    param = (D=D, D2=D2, invImD2=invImD2, D_ImD2=D_ImD2, tmp1=tmp1, tmp2=tmp2)

    ode = ODEProblem(bbm_bbm_periodic!, ηu0, tspan, param)
    
    saveat = range(tspan..., length=100)
    saved_values_baseline = SavedValues(eltype(D), SVector{6,eltype(D)})
    saving_baseline = SavingCallback(save_func_bbm_bbm_periodic, saved_values_baseline, saveat=saveat)
    saved_values_relaxation = SavedValues(eltype(D), SVector{6,eltype(D)})
    saving_relaxation = SavingCallback(save_func_bbm_bbm_periodic, saved_values_relaxation, saveat=saveat)
    relaxation = DiscreteCallback((u,t,integrator) -> true, relaxation!, save_positions=(false,false))
    cb_baseline = CallbackSet(saving_baseline)
    cb_relaxation = CallbackSet(relaxation, saving_relaxation)
    
    sol_relaxation = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_relaxation, tstops=saveat)
    flush(stdout)
    sol_baseline = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_baseline, tstops=saveat)
    flush(stdout)

    ηnum_baseline   = sol_baseline[end].x[1]
    unum_baseline   = sol_baseline[end].x[2]
    ηnum_relaxation = sol_relaxation[end].x[1]
    unum_relaxation = sol_relaxation[end].x[2]
    ηana = ηsol.(tspan[end], x, x[1], -x[1])
    uana = usol.(tspan[end], x, x[1], -x[1])
    @printf("Error in η (baseline):   %.3e\n", integrate(u->u^2, ηnum_baseline - ηana, D) |> sqrt)
    @printf("Error in η (relaxation): %.3e\n", integrate(u->u^2, ηnum_relaxation - ηana, D) |> sqrt)
    @printf("Error in u (baseline):   %.3e\n", integrate(u->u^2, unum_baseline - uana, D) |> sqrt)
    @printf("Error in u (relaxation): %.3e\n", integrate(u->u^2, unum_relaxation - uana, D) |> sqrt)
    @printf("Difference of baseline and relaxation in η: %.3e\n", 
        integrate(u->u^2, ηnum_baseline - ηnum_relaxation, D) |> sqrt)
    @printf("Difference of baseline and relaxation in u: %.3e\n", 
        integrate(u->u^2, unum_baseline - unum_relaxation, D) |> sqrt)

    sleep(0.1)
    fig_η, ax = plt.subplots(1, 1)
    plt.plot(x, η0, label=L"\eta^0")
    plt.plot(x, ηana, label=L"$\eta^\mathrm{ana}$")
    plt.plot(x, ηnum_baseline, label=L"$\eta^\mathrm{num}$ (non-conservative)")
    plt.plot(x, ηnum_relaxation, label=L"$\eta^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"\eta")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    fig_u, ax = plt.subplots(1, 1)
    plt.plot(x, u0, label=L"u^0")
    plt.plot(x, uana, label=L"$u^\mathrm{ana}$")
    plt.plot(x, unum_baseline, label=L"$u^\mathrm{num}$ (non-conservative)")
    plt.plot(x, unum_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"u")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    t_baseline = saved_values_baseline.t
    t_relaxation = saved_values_relaxation.t
    mass_η_baseline   = map(x->x[1], saved_values_baseline.saveval)
    mass_η_relaxation = map(x->x[1], saved_values_relaxation.saveval)
    mass_u_baseline   = map(x->x[2], saved_values_baseline.saveval)
    mass_u_relaxation = map(x->x[2], saved_values_relaxation.saveval)
    energy_baseline   = map(x->x[3], saved_values_baseline.saveval)
    energy_relaxation = map(x->x[3], saved_values_relaxation.saveval)
    quadratic_baseline   = map(x->x[4], saved_values_baseline.saveval)
    quadratic_relaxation = map(x->x[4], saved_values_relaxation.saveval)

    fig_invariants, ax = plt.subplots(1, 1)
    ax.set_yscale("symlog", linthreshy=1.0e-14)
    plt.plot(t_baseline,   mass_η_baseline   .- mass_η_baseline[1], 
                label=L"$\int \eta$ (non-conservative)",
                color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, mass_η_relaxation .- mass_η_relaxation[1], 
                label=L"$\int \eta$ (conservative)",
                color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   mass_u_baseline   .- mass_u_baseline[1], 
                label=L"$\int u$ (non-conservative)",
                color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, mass_u_relaxation .- mass_u_relaxation[1],
                label=L"$\int u$ (conservative)",
                color="#56B4E9", linestyle="--")
    plt.plot(t_baseline,   energy_baseline   .- energy_baseline[1], 
                label=L"$\int (\eta^2 + (1 + \eta) u^2)$ (non-conservative)",
                color="#E69F00", linestyle=":")
    plt.plot(t_relaxation, energy_relaxation .- energy_relaxation[1], 
                label=L"$\int (\eta^2 + (1 + \eta) u^2)$ (conservative)",
                color="#56B4E9", linestyle=":")
    plt.plot(t_baseline,   quadratic_baseline   .- quadratic_baseline[1], 
                label=L"$\int \eta (\mathrm{I} - \partial_x^2) u$ (non-conservative)",
                color="#E69F00", linestyle="-.")
    plt.plot(t_relaxation, quadratic_relaxation .- quadratic_relaxation[1], 
                label=L"$\int \eta (\mathrm{I} - \partial_x^2) u$ (conservative)",
                color="#56B4E9", linestyle="-.")
    plt.xlabel(L"t"); plt.ylabel("Change of Invariants")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_η_baseline   = map(x->x[5], saved_values_baseline.saveval)
    error_η_relaxation = map(x->x[5], saved_values_relaxation.saveval)
    error_u_baseline   = map(x->x[6], saved_values_baseline.saveval)
    error_u_relaxation = map(x->x[6], saved_values_relaxation.saveval)

    fig_error, ax = plt.subplots(1, 1)
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline, error_η_baseline, label=L"Error of $\eta$ (non-conservative)",
                color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_η_relaxation, label=L"Error of $\eta$ (conservative)",
                color="#56B4E9", linestyle="-")
    plt.plot(t_baseline, error_u_baseline, label=L"Error of $u$ (non-conservative)",
                color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, error_u_relaxation, label=L"Error of $u$ (conservative)",
                color="#56B4E9", linestyle="--")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    (; sol_relaxation, sol_baseline, saved_values_baseline, saved_values_relaxation, 
        fig_η, fig_u, fig_invariants, fig_error)
end


In [ ]:
# traveling wave solution with η>-1 obtained numerically
c, data = open("bbm_bbm_traveling_wave_init_c12_l90_N65536.txt", "r") do io
    line = readline(io)
    line = readline(io)
    line = readline(io)
    c = parse(Float64, line[7:end])
    data = readdlm(io, comments=true)
    c, data
end
x = range(data[1,1], data[end,1], length=size(data,1))
xmin = x[1]; xmax = x[end] + (x[2] - x[1])
η0 = data[:, 2]
u0 = data[:, 3]
η0itp = CubicSplineInterpolation((x,), η0, extrapolation_bc=Periodic())
u0itp = CubicSplineInterpolation((x,), u0, extrapolation_bc=Periodic())
get_c() = c
function ηsol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    η0itp(x_t)
end
function usol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    u0itp(x_t)
end


In [ ]:
println("c = ", get_c())
println("xmin = ", xmin)
println("xmax = ", xmax)
@show ηsol(0., xmax, xmin, xmax)
@show usol(0., xmax, xmin, xmax)
@show N = 2^8
@show dt = 0.5 * (xmax - xmin) / (N * get_c())

@show tspan = (0.0, (xmax-xmin)/(3*get_c()) + 100*(xmax-xmin)/get_c())
flush(stdout)

tol = 1.0e-5
adaptive = true
D = fourier_derivative_operator(xmin, xmax, N)
D2 = D^2

results = solve_ode_bbm_bbm_periodic(D, D2, tspan, Tsit5(), tol, dt, adaptive);

In [ ]:
ax = results.fig_error.axes[1]
t = [1.0e2, 1.0e4]
ax.plot(t, 3.0e-8 .* t.^2, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{2})", (5.0e2, 5.0e-2), color="gray")
ax.plot(t, 1.5e-7 .* t.^1, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{1})", (5.0e2, 3.0e-5), color="gray")

results.fig_error.savefig(joinpath(dirname(@__DIR__), "figures", "bbm_bbm_error.pdf"), 
    bbox_inches="tight")

# Conserving the quadratic functional

In [ ]:
function relaxation_functional(γ, ηunew, ηuold, param)
    @unpack D2, tmp1, tmp2 = param
    ηnew = ηunew.x[1]
    unew = ηunew.x[2]
    ηold = ηuold.x[1]
    uold = ηuold.x[2]
    
    @. tmp2 = (1-γ)*uold + γ*unew
    mul!(tmp1, D2, tmp2)
    @. tmp1 = ((1-γ)*ηold + γ*ηnew) * (tmp2 - tmp1)
    energy = integrate(tmp1, D)
end
function relaxation_functional(ηu, param)
    @unpack D, tmp1 = param
    η = ηu.x[1]
    u = ηu.x[2]
    
    mul!(tmp1, D2, u)
    @. tmp1 = η * (u - tmp1)
    integrate(tmp1, D)
end


function save_func_bbm_bbm_periodic(ηu, t, integrator)
    @unpack D, D2, tmp1 = integrator.p
    η = ηu.x[1]
    u = ηu.x[2]
    print(".")
    
    mass_η = integrate(η, D)
    mass_u = integrate(u, D)
    @. tmp1 = η^2 + (1 + η) * u^2
    energy = integrate(tmp1, D)
    
    mul!(tmp1, D2, u)
    @. tmp1 = η * (u - tmp1)
    quadratic = integrate(tmp1, D)
    
    x = grid(D)
    tmp1 .= ( ηsol.(t, x, x[1], -x[1]) .- η ).^2
    error_η = integrate(tmp1, D) |> sqrt
    tmp1 .= ( usol.(t, x, x[1], -x[1]) .- u ).^2
    error_u = integrate(tmp1, D) |> sqrt
    
    SVector(mass_η, mass_u, energy, quadratic, error_η, error_u)
end


function bbm_bbm_periodic!(dηu, ηu, param, t)
    @unpack D, invImD2, D_ImD2, tmp1, tmp2 = param
    η = ηu.x[1]
    u = ηu.x[2]
    dη = dηu.x[1]
    du = dηu.x[2]
    
    # quadratic functional and mass conservative semidiscretization
    @. tmp1 = -(u + η * u)
    if D_ImD2 === nothing
        ldiv!(tmp2, invImD2, tmp1)
        mul!(dη, D, tmp2)
    else
        mul!(dη, D_ImD2, tmp1)
    end

    if D_ImD2 === nothing
        ldiv!(tmp2, invImD2, η)
        mul!(du, D, tmp2)
    else
        mul!(du, D_ImD2, η)
    end
    mul!(tmp1, D, u)
    @. tmp1 = u * tmp1
    ldiv!(tmp2, invImD2, tmp1)
    @. du = -du - tmp2
    
    nothing
end

function solve_ode_bbm_bbm_periodic(D, D2, tspan, alg, tol, dt, adaptive)
    invImD2 = isa(D2, AbstractMatrix) ? factorize(I - D2) : I - D2
    D_ImD2 = if isa(D2, AbstractMatrix) 
        nothing
    else
        D / (I - D2)
    end
    
    x = grid(D)
    η0 = ηsol.(tspan[1], x, x[1], -x[1])
    u0 = usol.(tspan[1], x, x[1], -x[1])
    ηu0 = ArrayPartition(η0, u0)
    tmp1 = similar(u0); tmp2 = similar(tmp1)
    param = (D=D, D2=D2, invImD2=invImD2, D_ImD2=D_ImD2, tmp1=tmp1, tmp2=tmp2)

    ode = ODEProblem(bbm_bbm_periodic!, ηu0, tspan, param)
    
    saveat = range(tspan..., length=100)
    saved_values_baseline = SavedValues(eltype(D), SVector{6,eltype(D)})
    saving_baseline = SavingCallback(save_func_bbm_bbm_periodic, saved_values_baseline, saveat=saveat)
    saved_values_relaxation = SavedValues(eltype(D), SVector{6,eltype(D)})
    saving_relaxation = SavingCallback(save_func_bbm_bbm_periodic, saved_values_relaxation, saveat=saveat)
    relaxation = DiscreteCallback((u,t,integrator) -> true, relaxation!, save_positions=(false,false))
    cb_baseline = CallbackSet(saving_baseline)
    cb_relaxation = CallbackSet(relaxation, saving_relaxation)
    
    @time sol_relaxation = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_relaxation, tstops=saveat)
    flush(stdout)
    @time sol_baseline = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_baseline, tstops=saveat)
    flush(stdout)

    ηnum_baseline   = sol_baseline[end].x[1]
    unum_baseline   = sol_baseline[end].x[2]
    ηnum_relaxation = sol_relaxation[end].x[1]
    unum_relaxation = sol_relaxation[end].x[2]
    ηana = ηsol.(tspan[end], x, x[1], -x[1])
    uana = usol.(tspan[end], x, x[1], -x[1])
    @printf("Error in η (baseline):   %.3e\n", integrate(u->u^2, ηnum_baseline - ηana, D) |> sqrt)
    @printf("Error in η (relaxation): %.3e\n", integrate(u->u^2, ηnum_relaxation - ηana, D) |> sqrt)
    @printf("Error in u (baseline):   %.3e\n", integrate(u->u^2, unum_baseline - uana, D) |> sqrt)
    @printf("Error in u (relaxation): %.3e\n", integrate(u->u^2, unum_relaxation - uana, D) |> sqrt)
    @printf("Difference of baseline and relaxation in η: %.3e\n", 
        integrate(u->u^2, ηnum_baseline - ηnum_relaxation, D) |> sqrt)
    @printf("Difference of baseline and relaxation in u: %.3e\n", 
        integrate(u->u^2, unum_baseline - unum_relaxation, D) |> sqrt)

    sleep(0.1)
    fig_η, ax = plt.subplots(1, 1)
    plt.plot(x, η0, label=L"\eta^0")
    plt.plot(x, ηana, label=L"$\eta^\mathrm{ana}$")
    plt.plot(x, ηnum_baseline, label=L"$\eta^\mathrm{num}$ (non-conservative)")
    plt.plot(x, ηnum_relaxation, label=L"$\eta^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"\eta")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    fig_u, ax = plt.subplots(1, 1)
    plt.plot(x, u0, label=L"u^0")
    plt.plot(x, uana, label=L"$u^\mathrm{ana}$")
    plt.plot(x, unum_baseline, label=L"$u^\mathrm{num}$ (non-conservative)")
    plt.plot(x, unum_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"u")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    t_baseline = saved_values_baseline.t
    t_relaxation = saved_values_relaxation.t
    mass_η_baseline   = map(x->x[1], saved_values_baseline.saveval)
    mass_η_relaxation = map(x->x[1], saved_values_relaxation.saveval)
    mass_u_baseline   = map(x->x[2], saved_values_baseline.saveval)
    mass_u_relaxation = map(x->x[2], saved_values_relaxation.saveval)
    energy_baseline   = map(x->x[3], saved_values_baseline.saveval)
    energy_relaxation = map(x->x[3], saved_values_relaxation.saveval)
    quadratic_baseline   = map(x->x[4], saved_values_baseline.saveval)
    quadratic_relaxation = map(x->x[4], saved_values_relaxation.saveval)

    fig_invariants, ax = plt.subplots(1, 1)
    ax.set_yscale("symlog", linthreshy=1.0e-14)
    plt.plot(t_baseline,   mass_η_baseline   .- mass_η_baseline[1], label=L"$\int \eta$ (non-conservative)",
                color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, mass_η_relaxation .- mass_η_relaxation[1], label=L"$\int \eta$ (conservative)",
                color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   mass_u_baseline   .- mass_u_baseline[1], label=L"$\int u$ (non-conservative)",
                color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, mass_u_relaxation .- mass_u_relaxation[1], label=L"$\int u$ (conservative)",
                color="#56B4E9", linestyle="--")
    plt.plot(t_baseline,   energy_baseline   .- energy_baseline[1], label=L"$\int (\eta^2 + (1 + \eta) u^2)$ (non-conservative)",
                color="#E69F00", linestyle=":")
    plt.plot(t_relaxation, energy_relaxation .- energy_relaxation[1], label=L"$\int (\eta^2 + (1 + \eta) u^2)$ (conservative)",
                color="#56B4E9", linestyle=":")
    plt.plot(t_baseline,   quadratic_baseline   .- quadratic_baseline[1], label=L"$\int \eta (\mathrm{I} - \partial_x^2) u$ (non-conservative)",
                color="#E69F00", linestyle="-.")
    plt.plot(t_relaxation, quadratic_relaxation .- quadratic_relaxation[1], label=L"$\int \eta (\mathrm{I} - \partial_x^2) u$ (conservative)",
                color="#56B4E9", linestyle="-.")
    plt.xlabel(L"t"); plt.ylabel("Change of Invariants")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_η_baseline   = map(x->x[5], saved_values_baseline.saveval)
    error_η_relaxation = map(x->x[5], saved_values_relaxation.saveval)
    error_u_baseline   = map(x->x[6], saved_values_baseline.saveval)
    error_u_relaxation = map(x->x[6], saved_values_relaxation.saveval)

    fig_error, ax = plt.subplots(1, 1)
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline, error_η_baseline, label=L"Error of $\eta$ (non-conservative)",
                color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_η_relaxation, label=L"Error of $\eta$ (conservative)",
                color="#56B4E9", linestyle="-")
    plt.plot(t_baseline, error_u_baseline, label=L"Error of $u$ (non-conservative)",
                color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, error_u_relaxation, label=L"Error of $u$ (conservative)",
                color="#56B4E9", linestyle="--")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    (; sol_relaxation, sol_baseline, saved_values_baseline, saved_values_relaxation, 
        fig_η, fig_u, fig_invariants, fig_error)
end


In [ ]:
# traveling wave solution with η>-1 obtained numerically
c, data = open("bbm_bbm_traveling_wave_init_c12_l90_N65536.txt", "r") do io
    line = readline(io)
    line = readline(io)
    line = readline(io)
    c = parse(Float64, line[7:end])
    data = readdlm(io, comments=true)
    c, data
end
x = range(data[1,1], data[end,1], length=size(data,1))
xmin = x[1]; xmax = x[end] + (x[2] - x[1])
η0 = data[:, 2]
u0 = data[:, 3]
η0itp = CubicSplineInterpolation((x,), η0, extrapolation_bc=Periodic())
u0itp = CubicSplineInterpolation((x,), u0, extrapolation_bc=Periodic())
get_c() = c
function ηsol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    η0itp(x_t)
end
function usol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    u0itp(x_t)
end


In [ ]:
println("c = ", get_c())
println("xmin = ", xmin)
println("xmax = ", xmax)
@show ηsol(0., xmax, xmin, xmax)
@show usol(0., xmax, xmin, xmax)
@show N = 2^8
@show dt = 0.5 * (xmax - xmin) / (N * get_c())

@show tspan = (0.0, (xmax-xmin)/(3*get_c()) + 100*(xmax-xmin)/get_c())
flush(stdout)

tol = 1.0e-5
adaptive = true
D = fourier_derivative_operator(xmin, xmax, N)
D2 = D^2

results = solve_ode_bbm_bbm_periodic(D, D2, tspan, Tsit5(), tol, dt, adaptive);